# Dashboard

In [70]:
# export
import os
import numpy as np
import pandas as pd
import panel as pn
pn.extension('plotly')

import pyteomics.fasta
from io import StringIO
import plotly.graph_objects as go

from pepmap.importing import import_spectronaut_data
from pepmap.preprocessing import format_input_data
from pepmap.sequenceplot import plot_peptide_traces
from pepmap.uniprot_integration import uniprot_feature_dict
from pepmap.sequenceplot import uniprot_color_dict

In [71]:
path_to_folder_fasta_files = 'data'
full_fasta = None
full_uniprot = None

### Style of the dashboard

In [72]:
#export
css = '''
.bk.padding_widgets {
  padding: 2px;
}
.bk-root .bk-btn-default.bk-active {
  background-color: #f5a25d;
}
.bk.accordion {
  font-size: smaller;
}
.bk-root .bk-btn {
  font-size: 11px;
  padding: 6px 0px 6px 0px;
}
.bk.uniprot_options {
  margin: 16px 6px 16px 6px;
}
'''
pn.extension(raw_css=[css])

### Header

In [73]:
### Common widgets
header_titel = pn.pane.Markdown(
    '# Sequence Viewer', 
    align='center')
divider = pn.layout.Divider(
    align='center', 
    width=1500, 
    margin=(5, 0)
)

In [74]:
# Widgets
select_protein = pn.widgets.AutocompleteInput(
    name='Select protein:', 
    placeholder='Type first letters of the protein id...',
    min_characters=1)
select_organism = pn.widgets.Select(
    name='Select organism:', 
    value='Human', 
    options=['Human', 'Mouse', 'Arabidopsis'])
experimental_data = pn.widgets.FileInput(
    accept=".csv, .txt", 
    margin=20)
preprocessed_exp_data = pn.widgets.DataFrame(
    name='Exp_data')
visualize_button = pn.widgets.Button(
    name='Visualize Data', 
    button_type='primary', 
    css_classes=['padding_widgets'],
    height=40)

In [75]:
### Options
options_preprocessing_events = pn.widgets.CheckButtonGroup(
    name='Molecule processing', 
    value=['Chain', 'Peptide'], 
    options=['Chain', 'Initiator methionine', 'Peptide', 'Propeptide', 'Signal peptide', 'Transit peptide'],
    align='center')
options_PTMs = pn.widgets.CheckButtonGroup(
    name='Post-translational modification', 
    options=['Cross-link', 'Disulfide bond', 'Glycosylation', 'Lipidation', 'Modified residue'],
    align='center')
options_domains = pn.widgets.CheckButtonGroup(
    name='Family & Domain',  
    options=['Coiled coil', 'Compositional bias', 'Domain', 'Motif', 'Region', 'Repeat', 'Zinc finger'],
    align='center')
options_locations = pn.widgets.CheckButtonGroup(
    name='Subcellular location', 
    options=['Intramembrane', 'Topological domain', 'Transmembrane'],
    align='center')
options_functions = pn.widgets.CheckButtonGroup(
    name='Function', 
    options=['Active site', 'Binding site', 'Calcium binding', 'DNA binding', 'Metal binding', 'Nucleotide binding', 'Site'],
    align='center')
options_sequences = pn.widgets.CheckButtonGroup(
    name='Sequence', 
    options=['Alternative sequence', 'Natural variant', 'Non-adjacent residues', 'Non-standard residue', 
             'Non-terminal residue', 'Sequence conflict', 'Sequence uncertainty'],
    align='center', )
options_other = pn.widgets.CheckButtonGroup(
    name='Other options', 
    options=['Secondary structure', 'Mutagenesis'],
    align='center')

uniprot_options = pn.Accordion(
        options_preprocessing_events, 
        options_PTMs,
        options_domains,
        options_locations,
        options_functions,
        options_sequences,
        options_other,
        width = 800)

uniprot_options_tab = pn.Accordion(
    ('Uniprot Options', 
     uniprot_options
    ),
    width = 810,
    css_classes=['uniprot_options'],
    )

In [76]:
### PREPROCESSING
def upload_experimental_data():
    full_proteome_data = import_spectronaut_data(StringIO(str(experimental_data.value, "utf-8")))
    preprocessed_exp_data.value = format_input_data(
        df = full_proteome_data, 
        fasta = full_fasta, 
        modification_exp = r'\[.*?\]')
    select_protein.options = preprocessed_exp_data.value.unique_protein_id.unique().tolist()
        
def upload_organism_info():
    global full_fasta
    global full_uniprot
    if select_organism.value == 'Human':
        full_fasta = pyteomics.fasta.IndexedUniProt(os.path.join(path_to_folder_fasta_files, 
                                                                 "human.fasta"))
        full_uniprot = pd.read_csv(os.path.join(path_to_folder_fasta_files, 
                                                "preprocessed_uniprot_human.csv"))
    elif select_organism.value == 'Mouse':
        full_fasta = pyteomics.fasta.IndexedUniProt(os.path.join(path_to_folder_fasta_files, 
                                                                 "mouse.fasta"))
        full_uniprot = pd.read_csv(os.path.join(path_to_folder_fasta_files, 
                                                "preprocessed_uniprot_mouse.csv"))
    elif select_organism.value == 'Arabidopsis':
        full_fasta = pyteomics.fasta.IndexedUniProt(os.path.join(path_to_folder_fasta_files, 
                                                                 "arabidopsis_thaliana.fasta"))
        full_uniprot = pd.read_csv(os.path.join(path_to_folder_fasta_files, 
                                                "preprocessed_uniprot_arabidopsis.csv"))

In [77]:
### VISUALIZATION
@pn.depends(visualize_button.param.clicks, 
            select_organism.param.value, 
            experimental_data.param.value)
def visualize_data(clicks, org, exp):
    if clicks > 0:
        # preload the data
        upload_organism_info()
        upload_experimental_data()
        # create a layout
        app = pn.Column(
            pn.Row(
                pn.layout.VSpacer(width=150),
                select_protein,
                pn.layout.VSpacer(width=80),
                uniprot_options_tab,
                aligh='center'
            ),
            pn.layout.HSpacer(height=10),
            divider,
            pn.Pane(
                visualize_plot, 
                aligh='center', 
                width_policy='max',
                width=1500)
        )
        return app

@pn.depends(select_protein.param.value, 
            options_preprocessing_events.param.value, 
            options_PTMs.param.value, 
            options_domains.param.value, 
            options_locations.param.value,
            options_functions.param.value,
            options_sequences.param.value,
            options_other.param.value)
def visualize_plot(protein, *_):
    if protein:
        # combine selected uniprot options in one list
        uniprot_options_combined = sum([each.value for each in uniprot_options.objects if each.value], [])
        fig =  plot_peptide_traces(
            preprocessed_exp_data.value,
            name = experimental_data.filename.split('.')[0],
            protein = protein,
            fasta = full_fasta, 
            uniprot = full_uniprot,
            selected_features = [uniprot_feature_dict[each] for each in uniprot_options_combined],
            uniprot_feature_dict=uniprot_feature_dict, 
            uniprot_color_dict=uniprot_color_dict)
        return fig

In [80]:
layout = pn.Column(
    header_titel,
    divider,
    pn.WidgetBox(
        select_organism,
        experimental_data,
        margin=10,
        width=300,
        css_classes=['run_analysis']
    ),
    divider,
    visualize_button,
    divider,
    visualize_data
)

In [81]:
layout.servable()

Column
    [0] Markdown(str, align='center')
    [1] Divider(align='center', margin=(5, 0), width=1500)
    [2] WidgetBox(css_classes=['run_analysis'], margin=10, width=300)
        [0] Select(options=['Human', 'Mouse', ...], value='Human')
        [1] FileInput(accept='.csv, .txt', margin=20)
    [3] Divider(align='center', margin=(5, 0), width=1500)
    [4] Button(button_type='primary', css_classes=['padding_widgets'], height=40, name='Visualize Data')
    [5] Divider(align='center', margin=(5, 0), width=1500)
    [6] ParamFunction(function)